(line 17~18)
1. Normalization 68.13%
2. None Preprocessing 64%
3. Random Crop + Normalization 68.99%...我測:67.79%
4. Random Crop + Normalization + RandomHorizontalFlip 68.70%...我測:67.53%


In [ ]:
import torch
import torch.nn as nn
import torchvision
import matplotlib.pyplot as plt
import torch.utils.data as Data
from torch.autograd import Variable
import os

DOANLOAD_DATASET = True
LR = 0.001
BATCH_SIZE=256
EPOCH = 10
MODELS_PATH = './models'


train_transform = torchvision.transforms.Compose([
    # torchvision.transforms.RandomCrop(32, 4),
    # torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((.5, .5, .5), (.5, .5, .5))
])


test_transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((.5, .5, .5), (.5, .5, .5))
])

train_data = torchvision.datasets.CIFAR10(
    root='./cifar10',
    train=True,
    transform=train_transform,
    download=DOANLOAD_DATASET
)

test_data = torchvision.datasets.CIFAR10(
    root='./cifar10',
    train=False,
    transform=test_transform,
    download=DOANLOAD_DATASET
)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
data_loader = Data.DataLoader(
    dataset=train_data,
    batch_size=BATCH_SIZE,
    shuffle=True
)

In [ ]:
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [ ]:
class CNN(nn.Module):
  def __init__(self, num_classes: int):
    super(CNN, self).__init__()
    self.num_classes = num_classes

    # in[N, 3, 32, 32] => out[N, 16, 16, 16]
    self.conv1 = nn.Sequential(
        nn.Conv2d(
            in_channels=3,
            out_channels=16,
            kernel_size=5,
            stride=1,
            padding=2
        ),
        nn.ReLU(True),
        nn.MaxPool2d(kernel_size=2)
    )
    # in[N, 16, 16, 16] => out[N, 32, 8, 8]
    self.conv2 = nn.Sequential(
        nn.Conv2d(16, 32, 5, 1, 2),
        nn.ReLU(True),
        nn.MaxPool2d(2)

    )
    # in[N, 32 * 8 * 8] => out[N, 128]
    self.fc1 = nn.Sequential(
        nn.Linear(32 * 8 * 8, 128),
        nn.ReLU(True)
    )
    # in[N, 128] => out[N, 64]
    self.fc2 = nn.Sequential(
        nn.Linear(128, 64),
        nn.ReLU(True)
    )
    # in[N, 64] => out[N, 10]
    self.out = nn.Linear(64, self.num_classes)

  def forward(self, x):
    x = self.conv1(x)
    x = self.conv2(x)
    x = x.view(x.size(0), -1) # [N, 32 * 8 * 8]
    x = self.fc1(x)
    x = self.fc2(x)
    output = self.out(x)
    return output

In [ ]:
cnn = CNN(len(classes))
optimizer = torch.optim.Adam(cnn.parameters(), lr=LR)
loss_function = nn.CrossEntropyLoss()

for epoch in range(EPOCH):
  cnn.train()
  for step, (x, y) in enumerate(data_loader):
    b_x = Variable(x, requires_grad=False)
    b_y = Variable(y, requires_grad=False)
    out = cnn(b_x)
    loss = loss_function(out, b_y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if step % 100 == 0:
      print('Epoch: {} | Step: {} | Loss: {}'.format(epoch + 1, step, loss))
print('Done.')
if not os.path.exists(MODELS_PATH):
  os.mkdir(MODELS_PATH)
torch.save(cnn, os.path.join(MODELS_PATH, 'cnn_model.pt'))
print('Model Saved.')

Epoch: 1 | Step: 0 | Loss: 2.3111119270324707
Epoch: 1 | Step: 100 | Loss: 1.6786928176879883
Epoch: 2 | Step: 0 | Loss: 1.5457062721252441
Epoch: 2 | Step: 100 | Loss: 1.5607266426086426
Epoch: 3 | Step: 0 | Loss: 1.4526995420455933
Epoch: 3 | Step: 100 | Loss: 1.4075225591659546
Epoch: 4 | Step: 0 | Loss: 1.2629926204681396
Epoch: 4 | Step: 100 | Loss: 1.2712762355804443
Epoch: 5 | Step: 0 | Loss: 1.1919362545013428
Epoch: 5 | Step: 100 | Loss: 1.1716136932373047
Epoch: 6 | Step: 0 | Loss: 1.0951350927352905
Epoch: 6 | Step: 100 | Loss: 1.0656310319900513
Epoch: 7 | Step: 0 | Loss: 1.1962224245071411
Epoch: 7 | Step: 100 | Loss: 1.1916143894195557
Epoch: 8 | Step: 0 | Loss: 1.1001381874084473
Epoch: 8 | Step: 100 | Loss: 1.1029504537582397
Epoch: 9 | Step: 0 | Loss: 1.0669103860855103
Epoch: 9 | Step: 100 | Loss: 0.8956103920936584
Epoch: 10 | Step: 0 | Loss: 1.0907150506973267
Epoch: 10 | Step: 100 | Loss: 0.891814112663269
Done.
Model Saved.


In [ ]:
test_loader = Data.DataLoader(
    dataset=test_data,
    batch_size=test_data.data.shape[0],
    shuffle=False
)
test_x, test_y = next(iter(test_loader))
print(test_x.shape)
print(test_y.shape)

cnn.eval()
prediction = torch.argmax(cnn(test_x), 1)
acc = torch.eq(prediction, test_y)
print('Accuracy: {:.2%}'.format((torch.sum(acc) / acc.shape[0]).item()))

torch.Size([10000, 3, 32, 32])
torch.Size([10000])
Accuracy: 67.53%
